In [1]:
import numpy as np
import pandas as pd

In [4]:
train_df = pd.read_pickle('./data/train_df.pkl')
eval_df = pd.read_pickle('./data/eval_df.pkl')

In [14]:
def calculate_rmse(x, y):
    return np.sum(np.absolute(x - y))/x.shape[0]

In [35]:
def calculate_smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.nanmean(diff)

In [13]:
np.array([1,2,3])[:-1]

array([1, 2])

In [36]:
# USE PREVIOUS DATAPOINT AS PREDICT VAL
baseline_rmse = []
for gh in eval_df.geohash6.unique():
    demand = eval_df[eval_df.geohash6 == gh].demand.values
    
    rmse_t_1 = calculate_rmse(demand[:-5], demand[1:-4])
    rmse_t_2 = calculate_rmse(demand[:-5], demand[2:-3])
    rmse_t_3 = calculate_rmse(demand[:-5], demand[3:-2])
    rmse_t_4 = calculate_rmse(demand[:-5], demand[4:-1])
    rmse_t_5 = calculate_rmse(demand[:-5], demand[5:])
    
    smape_t_1 = calculate_smape(demand[:-5], demand[1:-4])
    smape_t_2 = calculate_smape(demand[:-5], demand[2:-3])
    smape_t_3 = calculate_smape(demand[:-5], demand[3:-2])
    smape_t_4 = calculate_smape(demand[:-5], demand[4:-1])
    smape_t_5 = calculate_smape(demand[:-5], demand[5:])
    
    baseline_rmse.append({
        'geohash6': gh,
        'rmse(t+1)': rmse_t_1,
        'rmse(t+2)': rmse_t_2,
        'rmse(t+3)': rmse_t_3,
        'rmse(t+4)': rmse_t_4,
        'rmse(t+5)': rmse_t_5,
        'smape(t+1)': smape_t_1,
        'smape(t+2)': smape_t_2,
        'smape(t+3)': smape_t_3,
        'smape(t+4)': smape_t_4,
        'smape(t+5)': smape_t_5,
    })

/anaconda3/envs/grab/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [37]:
baseline_rmse_df = pd.DataFrame(baseline_rmse)

In [38]:
baseline_rmse_df.describe()

,rmse(t+1),rmse(t+2),rmse(t+3),rmse(t+4),rmse(t+5),smape(t+1),smape(t+2),smape(t+3),smape(t+4),smape(t+5)
count,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000
mean,0.013839,0.016873,0.018943,0.020778,0.022645,42.125687,47.338836,50.168430,52.528390,54.806699
std,0.012497,0.015914,0.018763,0.021544,0.024461,22.128115,24.171733,24.997878,25.658239,26.226822
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.003566,0.004060,0.004247,0.004377,0.004524,26.400613,30.849301,32.858724,34.984050,37.107088
50%,0.011341,0.013122,0.014106,0.014763,0.015361,43.932744,50.104985,53.953105,57.626416,61.319348
75%,0.020943,0.025077,0.027514,0.029703,0.032105,59.748340,66.397953,69.274069,71.613339,74.366964
max,0.066992,0.084902,0.103579,0.121470,0.140604,97.258248,104.714888,105.229671,111.130097,113.517759


In [39]:
baseline_rmse_df.to_csv('./baseline_using_prev_datapoint.csv', index=False)